In [177]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [178]:
df = pd.read_csv("data/heart2.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,0
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,1
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,1
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,0
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,0


In [179]:
df.dtypes

age           int64
sex           int64
cp           object
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca          float64
thal         object
target        int64
dtype: object

In [180]:
def cat_codes(df, col_name):
    df[col_name] = pd.Categorical(df[col_name])
    df[col_name] = df[col_name].cat.codes

cat_colums = ["cp"]
for col in cat_colums:
    cat_codes(df, col)

df['thal'] = df['thal'].astype('str')

In [181]:
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,2,150,0,2.3,3,0.0,fixed,0
1,67,1,0,160,286,0,2,108,1,1.5,2,3.0,normal,1
2,67,1,0,120,229,0,2,129,1,2.6,2,2.0,reversable,1
3,37,1,1,130,250,0,0,187,0,3.5,3,0.0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,normal,0


In [182]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


In [183]:
def df_to_dataset(df, shuffle=True, batch_size=32):
    df = df.copy()
    labels = df.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))
    ds = ds.batch(batch_size)
    return ds

In [184]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [185]:
for feature_batch, label_batch in train_ds.take(1):
    print("Every feature:", list(feature_batch.keys()))
    print("A batch of ages:", feature_batch['age'])
    print("A batch of targets:", label_batch)

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([41 65 35 51 61], shape=(5,), dtype=int32)
A batch of targets: tf.Tensor([1 0 1 0 1], shape=(5,), dtype=int32)


In [186]:
example_batch = next(iter(train_ds))[0]

In [187]:
def demo(feature_column):
    feature_layer = layers.DenseFeatures(feature_column)
    print(feature_layer(example_batch).numpy())

In [188]:
age = feature_column.numeric_column("age")
demo(age)

[[61.]
 [56.]
 [61.]
 [51.]
 [61.]]


In [189]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
demo(age_buckets)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [190]:
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)

[[0. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [191]:
tha1_embedding = feature_column.embedding_column(thal, dimension=8)
demo(tha1_embedding)

[[ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.00800326 -0.00177795  0.12067082  0.5695621  -0.11213961  0.06052765
   0.10184006 -0.5915288 ]
 [ 0.00800326 -0.00177795  0.12067082  0.5695621  -0.11213961  0.06052765
   0.10184006 -0.5915288 ]
 [ 0.00800326 -0.00177795  0.12067082  0.5695621  -0.11213961  0.06052765
   0.10184006 -0.5915288 ]
 [ 0.00800326 -0.00177795  0.12067082  0.5695621  -0.11213961  0.06052765
   0.10184006 -0.5915288 ]]


In [192]:
thal_hashed = feature_column.categorical_column_with_hash_bucket('thal', hash_bucket_size=1000)
demo(feature_column.indicator_column(thal_hashed))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [193]:
cross_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
demo(feature_column.indicator_column(cross_feature))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [199]:
feas = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']
def get_normalization_parameters(traindf, features):
    """Get the normalization parameters (E.g., mean, std) for traindf for 
    features. We will use these parameters for training, eval, and serving."""

    def _z_score_params(column):
        mean = traindf[column].mean()
        std = traindf[column].std()
        return {'mean': mean, 'std': std}

    normalization_parameters = {}
    for column in features:
        normalization_parameters[column] = _z_score_params(column)
    return normalization_parameters

normalization_parameters = get_normalization_parameters(train, feas)

def _numeric_column_normalized(column_name, normalizer_fn):
    return tf.feature_column.numeric_column(column_name, normalizer_fn=normalizer_fn)

def _make_zscaler(mean, std):
    def zscaler(col):
        return (col - mean)/std
    return zscaler

# Define your feature columns
def create_feature_cols(features, use_normalization):
    """Create feature columns using tf.feature_column. 
    
    This function will get executed during training, evaluation, and serving."""
    def normalize_column(col):  # Use mean, std defined below.
        return (col - mean)/std
    normalized_feature_columns = []
    for column_name in features:
        normalizer_fn = None
        if use_normalization:
            column_params = normalization_parameters[column_name]
            mean = column_params['mean']
            std = column_params['std']
            normalizer_fn = _make_zscaler(mean, std)
        normalized_feature_columns.append(_numeric_column_normalized(column_name,
                                                                     normalizer_fn))
    return normalized_feature_columns

In [195]:

feature_columns = create_feature_cols(features=feas, use_normalization=True)
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

cross_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
cross_feature = feature_column.indicator_column(cross_feature)
feature_columns.append(cross_feature)

In [196]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [197]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy',
    metrics=['accuracy'],
    run_eagerly=True    
)

model.fit(train_ds, validation_data=val_ds, epochs=100)

Epoch 1/100
7/7 [==============================] - 0s 38ms/step - loss: nan - accuracy: 0.5492 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/100
7/7 [==============================] - 0s 29ms/step - loss: nan - accuracy: 0.5492 - val_loss: nan - val_accuracy: 0.5102
Epoch 3/100
7/7 [==============================] - 0s 29ms/step - loss: nan - accuracy: 0.5492 - val_loss: nan - val_accuracy: 0.5102
Epoch 4/100
7/7 [==============================] - 0s 31ms/step - loss: nan - accuracy: 0.5492 - val_loss: nan - val_accuracy: 0.5102
Epoch 5/100
7/7 [==============================] - 0s 34ms/step - loss: nan - accuracy: 0.5492 - val_loss: nan - val_accuracy: 0.5102
Epoch 6/100
7/7 [==============================] - 0s 42ms/step - loss: nan - accuracy: 0.5492 - val_loss: nan - val_accuracy: 0.5102
Epoch 7/100
7/7 [==============================] - 0s 32ms/step - loss: nan - accuracy: 0.5492 - val_loss: nan - val_accuracy: 0.5102
Epoch 8/100
7/7 [==============================] - 0s 31ms

In [198]:
loss , accuracy = model.evaluate(test_ds)
print("loss: {} accuracy: {}".format(loss, accuracy))

2/2 [==============================] - 0s 35ms/step - loss: nan - accuracy: 0.5410
loss: nan accuracy: 0.5409836173057556
